## meta

In [1]:
from src.etl import *
from src.utils import *
import yaml
from collections import Counter
from collections import defaultdict

In [2]:
params['data']['image_size']

224

In [3]:
META = etl()

etl: META from src 0.. 
etl: META from src 1.. 
etl: META from src 2.. 
etl: META from src 3.. 
etl: META from src 4.. 
done!


In [4]:
META

,patient,img,label,src,Unnamed: 0
0,2,./data/source/covid-chestxray-dataset-master/i...,covid,0,NaN
1,2,./data/source/covid-chestxray-dataset-master/i...,covid,0,NaN
2,2,./data/source/covid-chestxray-dataset-master/i...,covid,0,NaN
3,2,./data/source/covid-chestxray-dataset-master/i...,covid,0,NaN
4,4,./data/source/covid-chestxray-dataset-master/i...,covid,0,NaN
...,...,...,...,...,...
22044,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,./data/source/rsna-pneumonia-detection-challen...,pneumonia,4,NaN
22045,c1edf42b-5958-47ff-a1e7-4f23d99583ba,./data/source/rsna-pneumonia-detection-challen...,normal,4,NaN
22046,c1f6b555-2eb1-4231-98f6-50a963976431,./data/source/rsna-pneumonia-detection-challen...,normal,4,NaN
22047,c1f7889a-9ea9-4acb-b64c-b737c929599a,./data/source/rsna-pneumonia-detection-challen...,pneumonia,4,NaN


In [16]:
print('label distribution: ', Counter(META.label), sep='\n')

label distribution: 
Counter({'pneumonia': 11092, 'normal': 10340, 'covid': 617})


## train / test split

In [17]:
META = dataset_split(META)
META.head()

,imgid,patient,img,label,src,train
0,./data/train\covid\0.jpeg,2,./data/source/covid-chestxray-dataset-master/i...,0,0,1
1,./data/test\covid\1.jpeg,2,./data/source/covid-chestxray-dataset-master/i...,0,0,0
2,./data/train\covid\2.jpeg,2,./data/source/covid-chestxray-dataset-master/i...,0,0,1
3,./data/train\covid\3.jpeg,2,./data/source/covid-chestxray-dataset-master/i...,0,0,1
4,./data/test\covid\4.jpeg,4,./data/source/covid-chestxray-dataset-master/i...,0,0,0


In [18]:
print('label integer map: ', params['train']['labelmap'], sep='\n', end='\n\n')
for ds in (0, 1):
    print('{0} data from covid datasets: '.format({0: 'test', 1: 'train'}[ds]), Counter(META[META.train==ds].label), sep='\n', end='\n\n')

label integer map: 
{'covid': 0, 'normal': 1, 'pneumonia': 2}

test data from covid datasets: 
Counter({2: 2773, 1: 2585, 0: 155})

train data from covid datasets: 
Counter({2: 8319, 1: 7755, 0: 462})



In [19]:
# cache image meta
if not os.path.isdir(SAVE_PATH):
    os.makedirs(SAVE_PATH)
with open(os.path.join(SAVE_PATH, 'meta'), 'wb') as pickle_file:
    pickle.dump(META, pickle_file, pickle.HIGHEST_PROTOCOL)
    print(f"META data saved at {os.path.join(SAVE_PATH, 'meta')}")

META data saved at ./data/meta


In [20]:
%%time
if False:
    # -------------------------------------------------------------------------------------------------------------------
    # imagge file to matrix  
    # NOTE: this needs model to load all data at once into cache which take up too much storage, 
    #       making computing inefficient, NOT GOOD
    # -------------------------------------------------------------------------------------------------------------------
    # train_data / test_data structure: 
    #                        {'covid': {'data': list of size*size*3 numpy array, 'label': corresponding list of integer},
    #                        '!covid': {'data': list of size*size*3 numpy array, 'label': corresponding list of integer}}
    TRAIN_DATA, TEST_DATA = datafromfile(META)
    # a sanity check
    print(f"train covid sample #: {len(TRAIN_DATA['covid']['label'])}")
    print(f"train !covid sample #: {len(TRAIN_DATA['!covid']['label'])}")
    print(f"test  covid sample #: {len(TEST_DATA['covid']['label'])}")
    print(f"test  !covid sample #: {len(TEST_DATA['!covid']['label'])}")
    # cache image data
    with open(os.path.join(SAVE_PATH, 'train.data'), 'wb') as pickle_file:
        pickle.dump(TRAIN_DATA, pickle_file, pickle.HIGHEST_PROTOCOL)
        print(f"Image data saved at {os.path.join(SAVE_PATH, 'train.data')}")
    with open(os.path.join(SAVE_PATH, 'test.data'), 'wb') as pickle_file:
        pickle.dump(TEST_DATA, pickle_file, pickle.HIGHEST_PROTOCOL)
        print(f"Image data saved at {os.path.join(SAVE_PATH, 'test.data')}")
else:
    # merge imgaes together instead, and transform image to batches of matrix during training
    mergesoure(META)

Wall time: 10min 21s


In [21]:
# sanity check for image shape, they need to consistent with model input
import cv2
import glob
import numpy as np
for i in np.random.choice(META.index, 10):
    print(i, cv2.imread(META.loc[i, 'imgid']).shape)

6642 (224, 224, 3)
3297 (224, 224, 3)
10115 (224, 224, 3)
14285 (224, 224, 3)
1258 (224, 224, 3)
21797 (224, 224, 3)
13709 (224, 224, 3)
21755 (224, 224, 3)
8634 (224, 224, 3)
4543 (224, 224, 3)
